#### 1)파이썬 3.6버전
#### 2)비주얼 스튜디오 설치(c++있어야함)
#### 3)!pip install cmake
#### 4)dlib 설치: conda install -c conda-forge dlib 실행
#### 5)face_recognition 설치: conda install -c conda-forge face_recognition 실행

### 필요한 패키지 불러오기

In [ ]:
import os
import numpy as np
import pandas as pd
from PIL import Image

import cv2
from matplotlib import pyplot as plt

import glob

import face_recognition
import pyttsx3

import telegram

### 얼굴 이미지 노이즈 제거 후 크롭하여 저장하기

#### 경로 설정

In [ ]:
uncropped_path = 'C:/Users/Chanhui1/uncropped_4/' #전처리 되지 않은 이미지 위치
blured_path = 'C:/Users/Chanhui1/blured_4/' #노이즈 제거한 이미지 저장할 위치
cropped_path = 'C:/Users/Chanhui1/cropped_4/' #노이즈 제거 후 크롭한 이미지 저장할 위치(전처리 완료된 이미지 저장 위치)

#### 노이즈 제거

In [ ]:
plt.rcParams["figure.figsize"] = (16,9)

path = glob.glob(uncropped_path + '*.jpg') #전처리 되지 않은 데이터 위치 + 형식

cv_img_blur = []
ids_blur = []
num = 0

for img in path:
    n=cv2.imread(img)
    id = str(os.path.split(img)[-1].split(".")[0])
    cv_img_blur.append(n)
    ids_blur.append(id)
    
for img in cv_img_blur:
    final_image = cv2.bilateralFilter(img, 13, 25, 9) #노이즈 제거 적용
    cv2.imwrite(blured_path + ids_blur[num] + '.jpg',final_image) #blured_path에 노이즈 제거 이미지 저장
    num = num + 1

#### 노이즈 제거 한 이미지 크롭

In [ ]:
plt.rcParams["figure.figsize"] = (16,9)

cascade_file = ".\\haarcascade_frontface.xml"


path = glob.glob(blured_path + '*.jpg')

cv_img = []
ids = []

for img in path:
    n=cv2.imread(img)
    id = str(os.path.split(img)[-1].split(".")[0])
    cv_img.append(n)
    ids.append(id)
    
num = 0

for i in cv_img: # for문으로 cv_img 리스트에 있는 이미지 데이터 하나씩 꺼내기#
    image_gs = cv2.cvtColor(i, cv2.COLOR_BGR2GRAY)
    cascade = cv2.CascadeClassifier(cascade_file)
    face_list = cascade.detectMultiScale(image_gs, minSize = (30,30)) #얼굴좌표#
    print(face_list) 
    
    if len(face_list) == 1: # 얼굴좌표가 인식됐다면#
        image_face = i.copy() # image_face에 이미지 데이터 대입#
        color = (0,0,255) # 빨강
        for face in face_list:
            x,y,w,h = face
            cv2.rectangle(image_face,(x,y),(x+w,y+h), color, thickness = 3)   #image_face에 얼굴 인식 좌표 그리기#
            rectangle_image = cv2.cvtColor(image_face, cv2.COLOR_BGR2RGB)
            crop_image = rectangle_image[y:y+h, x:x+w]  #crop_image에 얼굴만 크롭한 이미지 넣기#
            final_image = cv2.cvtColor(crop_image, cv2.COLOR_RGB2BGR)
            cv2.imwrite(cropped_path+str(num)+'.jpg',final_image)
            num = num + 1

[[1092 1775  781  781]]
[[1071 1390  804  804]]
[[ 662 1322 1360 1360]]
[[1217 1821  645  645]]


## 웹캠으로 얼굴 인식하기

#### 소리 설정하기

In [ ]:
# TTS 엔진 초기화
engine = pyttsx3.init()

# 말하는 속도
engine.setProperty('rate',180)
rate = engine.getProperty('rate')

# 소리 크기
engine.setProperty('volume',0.7)
volume = engine.getProperty('volume')

# 목소리
voices = engine.getProperty('voices')
engine.setProperty('voice',voices[0].id) #남성
#engine.setProperty('voice',voices[1].id) #여성

#### 텔레그램 설정하기

In [ ]:
chat_token = "1978535694:AAGWsSsB8hKaenqYEexJ8_MK2nLm6AJfyuE" #텔레그램-> botFather-> /newbot->id만들면=>토큰 받을 수 있음 
chat = telegram.Bot(token = chat_token)
updates = chat.getUpdates()
for u in updates:
    print(u.message['chat']['id'])

bot = telegram.Bot(token = "1978535694:AAGWsSsB8hKaenqYEexJ8_MK2nLm6AJfyuE")

chat_id = '1888582466' #https://api.telegram.org/bot[Token 값]/getUpdates확인해서 id확인

### 얼굴 인식 함수 만들기

In [ ]:
# 얼굴 인식(웹캠)
# 웹캠 0(기본값)에 대한 레퍼런스 가져오기



def webcam():
    
    
    video_capture = cv2.VideoCapture('C:/Users/Chanhui1/IMG_6616.mp4') # 웹캠 출력(웹캠: 0, 비디오 파일: 비디오 파일 위치, 파일 이름 대입하면 됨.)

    path = glob.glob(cropped_path + '*.jpg') # 크롭된 이미지 있는 위치 + 이미지 형식

    ids = []
    
    # 인식시킬 사진을 로드하고 인식시키기
    for img in path:
        id = str(os.path.split(img)[-1].split(".")[0])
        ids.append(id)

    known_face_encodings = []

    for name in ids:
        image = face_recognition.load_image_file(cropped_path + name + ".jpg")
        face_encoding = face_recognition.face_encodings(image)[0]
        known_face_encodings.append(face_encoding)

    # 특정 변수 초기값 선언
    face_locations = []
    face_encodings = []
    process_this_frame = True
    face_names_acc_on=[] # face_names 누적 리스트_승차용 리스트
    face_names_acc_off = [] # 누적 리스트_하차용 리스트
    change_button = True # 승차, 하차 버튼 논리값

    # 목소리내는데 쓰일 리스트 갯수 변수
    len_get_on = len(face_names_acc_on) # 승차리스트 
    len_get_out = len(face_names_acc_off) # 하차리스트

    get_on = str(0)
    get_off = str(0)
    total = str(0)

    while True:
        # 비디오의 한 프레임 잡기
        ret, frame = video_capture.read()

        # 더 빠른 얼굴 인식 처리를 위해 비디오 프레임 크기를 1/4 크기로 조정
        small_frame = cv2.resize(frame, (0, 0), fx=0.25, fy=0.25) #비디오 대입시 오류 발생하는 부분.

        # 이미지를 BGR 색상(OpenCV 사용)에서 RGB 색상(face_recognition 사용)으로 변환
        rgb_small_frame = cv2.cvtColor(small_frame, cv2.COLOR_BGR2RGB)

        # 시간을 절약하기 위해 비디오의 다른 모든 프레임만 처리
        if process_this_frame:
            # 비디오의 현재 프레임에서 모든 얼굴 위치와 인코딩 찾기
            face_locations = face_recognition.face_locations(rgb_small_frame)
            face_encodings = face_recognition.face_encodings(rgb_small_frame, face_locations)

            face_names = []
            for face_encoding in face_encodings:
                # See if the face is a match for the known face(s)
                matches = face_recognition.compare_faces(known_face_encodings, face_encoding)
                name = "Unknown"

                # known_face_encodings에서 일치하는 항목이 발견되면 첫 번째 항목만 사용
                # if True in matches:
                #     first_match_index = matches.index(True)
                #     name = known_face_names[first_match_index]

                # 또는 새 얼굴까지의 거리가 가장 작은 알려진 얼굴을 사용
                face_distances = face_recognition.face_distance(known_face_encodings, face_encoding)
                best_match_index = np.argmin(face_distances)
                if matches[best_match_index]:
                    name = ids[best_match_index]

                face_names.append(name)

                # Unkown일 땐 카운트하지 않기
                if name != 'Unknown':
                    if change_button:
                        face_names_acc_on.append(name.split('_')[0]) # 숫자 제외한 이름 추가
                    else:
                        face_names_acc_off.append(name.split('_')[0])

                
                # 현재 승차인원 리스트
                face_names_acc_total = list(set(face_names_acc_on) - set(face_names_acc_off)) #차량 탑승 인원 리스트

                # 승차버튼: a, 하차버튼: b
                if cv2.waitKey(1) == ord('a'):
                    change_button = True

                if cv2.waitKey(1) == ord('b'):
                    change_button = False

                    
                
                # '승차입니다', '하차입니다' 말하고 텔레그램으로 문자 보내기
                if change_button == True:
                    if len_get_on != len(set(face_names_acc_on)):
                        engine.say(name.split('_')[0]+'  승차입니다.')
                        engine.runAndWait() #말 다할때까지 대기
                        engine.stop() #끝
                        len_get_on = len(set(face_names_acc_on))
                        
                        bot.sendMessage(chat_id=chat_id, text=name.split('_')[0]+'승차 했습니다.')
                    else:
                        continue

                else:
                    if len_get_out != len(set(face_names_acc_off)):
                        engine.say(name.split('_')[0]+'  하차입니다.')
                        engine.runAndWait() #말 다할때까지 대기
                        engine.stop() #끝
                        len_get_out = len(set(face_names_acc_off))
                        
                        bot.sendMessage(chat_id=chat_id, text=name.split('_')[0]+'하차 했습니다.')
                    else:
                        continue


        process_this_frame = not process_this_frame #원본엔 이게 있었는데 왜 있는지 모르겠습니다.

        font = cv2.FONT_HERSHEY_DUPLEX
        
        # 결과 표시
        for (top, right, bottom, left), name in zip(face_locations, face_names):
            # 감지한 프레임이 1/4 크기로 축소되었기 때문에 얼굴 위치를 백업
            top *= 4
            right *= 4
            bottom *= 4
            left *= 4

            # 얼굴 주위에 박스 그림
            cv2.rectangle(frame, (left, top), (right, bottom), (0, 0, 255), 2)

            # 얼굴 아래에 이름으로 라벨 표시
            cv2.rectangle(frame, (left, bottom - 35), (right, bottom), (0, 0, 255), cv2.FILLED)
            cv2.putText(frame, name.split('_')[0], (left + 6, bottom - 6), font, 1.0, (255, 255, 255),1)

            get_on = str(len(set(face_names_acc_on)))
            get_off = str(len(set(face_names_acc_off)))
            total = str(len(set(face_names_acc_total)))



        # 승하차 여부 표시 (승차 : get on, 하차 : get off, 총 인원 : total)
        
        cv2.rectangle(frame, (0, 300), (200, 480), (104,213,244), cv2.FILLED)
        cv2.putText(frame, 'get on: '+get_on,(20,370),font,1.0,(0,0,0))
        cv2.putText(frame, 'get off: '+get_off,(20,400),font,1.0,(0,0,0))
        cv2.putText(frame, 'total: '+total,(20,450),font,1.0,(0,0,0))
        
        # 승하차 모드 표시
        if change_button:
            cv2.putText(frame, 'mode: on',(20,330),font,1.0,(0,0,0))
        else:
            cv2.putText(frame, 'mode: off',(20,330),font,1.0,(0,0,0))
            

        # 결과 보여주기
        cv2.imshow('Video', frame)


        # 'c' 누르면 '현재 인원 ?? 명입니다.' 말하기
        if cv2.waitKey(1) == ord('c'):
            engine.say('현재 인원'+total+'명 입니다.')
            engine.runAndWait() #말 다할때까지 대기
            engine.stop() #끝


        # 'q' 누르면 종료
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break


    # 웹캠으로 내보내기
    video_capture.release()
    cv2.destroyAllWindows()

In [ ]:
webcam()

#### 실행하면 기본값이 승차모드고, 하차 버튼(b)누르면 하차 모드로 변경됩니다. 다소 인식을 잘 못 하는 경향이 있지만 여러 번 누르면 됩니다....
승차버튼: a, 
하차버튼: b, 
현재 인원 알림 버튼: c, 
종료버튼: q